# Tool Calling = foundation of modern agents

# CAN you see this?

In [9]:
from openai import OpenAI

client = OpenAI()

In [10]:
response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {"role": "user", "content": "Hi"}
    ]
)

response.choices[0].message.content

'Hi — how can I help you today?'

basic LLM response

## tool calling

output of an LLM = > converting that into something we can EXECUTE

In [11]:
# The old school way of doing tool calling was raw dogging it in the prompt

user_prompt = "Send email to lucasbnsoares@hotmail.com, containing 3 things you think are essential for someone to be a life long learner"

prompt_with_tool_call_instructions = f"""
You are a helpful assistant that can use tools to help the user.

Here are the tools available to you:

send_email(to, subject, body)
- send an email to the user
- to is a string of the email address of the receiver
- subject is a string of the subject of the email
- body is a string of the body of the email

If the task at hand requires you to use a tool, please format your output as valid JSON, for example:

{{
  "name": "send_email",
  "parameters": {{
    "to": "...",
    "subject": "...",
    "body": "..."
  }}
}}

Here is the user's prompt:

{user_prompt}
"""

response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {"role": "user", "content": prompt_with_tool_call_instructions}
    ],
)

output_tool_call = response.choices[0].message.content

output_tool_call

'{\n  "name": "send_email",\n  "parameters": {\n    "to": "lucasbnsoares@hotmail.com",\n    "subject": "Three essentials for being a lifelong learner",\n    "body": "Hi Lucas,\\n\\nHere are three things I think are essential for someone to become and stay a lifelong learner:\\n\\n1) Curiosity and a growth mindset\\n   - Stay curious and ask questions. Treat challenges and mistakes as opportunities to grow rather than signs of failure.\\n\\n2) Consistent, deliberate practice and reflection\\n   - Break skills into small, manageable goals, practice regularly with focused effort, and reflect on what worked and what didn’t so you can improve.\\n\\n3) Metacognition and effective learning strategies\\n   - Know how you learn best and use proven techniques (active recall, spaced repetition, summarization, and good note systems) to retain and apply knowledge.\\n\\nHope you find these helpful. If you want, I can expand on any of these or suggest practical routines and tools to put them into pra

In [13]:
import json

tool_call_payload = json.loads(output_tool_call)



print(tool_call_payload["name"])
print(tool_call_payload["parameters"])

send_email
{'to': 'lucasbnsoares@hotmail.com', 'subject': 'Three essentials for being a lifelong learner', 'body': 'Hi Lucas,\n\nHere are three things I think are essential for someone to become and stay a lifelong learner:\n\n1) Curiosity and a growth mindset\n   - Stay curious and ask questions. Treat challenges and mistakes as opportunities to grow rather than signs of failure.\n\n2) Consistent, deliberate practice and reflection\n   - Break skills into small, manageable goals, practice regularly with focused effort, and reflect on what worked and what didn’t so you can improve.\n\n3) Metacognition and effective learning strategies\n   - Know how you learn best and use proven techniques (active recall, spaced repetition, summarization, and good note systems) to retain and apply knowledge.\n\nHope you find these helpful. If you want, I can expand on any of these or suggest practical routines and tools to put them into practice.\n\nBest,\nYour AI assistant'}


In [15]:
def send_email(to, subject, body):
    # toy example
    print(f"Sending email to {to} with subject {subject} and body {body}")

function_to_call = eval(tool_call_payload["name"])
function_parameters = tool_call_payload["parameters"]
# calling the function send_email with the right parameters selected by the LLM!
function_to_call(**function_parameters)

Sending email to lucasbnsoares@hotmail.com with subject Three essentials for being a lifelong learner and body Hi Lucas,

Here are three things I think are essential for someone to become and stay a lifelong learner:

1) Curiosity and a growth mindset
   - Stay curious and ask questions. Treat challenges and mistakes as opportunities to grow rather than signs of failure.

2) Consistent, deliberate practice and reflection
   - Break skills into small, manageable goals, practice regularly with focused effort, and reflect on what worked and what didn’t so you can improve.

3) Metacognition and effective learning strategies
   - Know how you learn best and use proven techniques (active recall, spaced repetition, summarization, and good note systems) to retain and apply knowledge.

Hope you find these helpful. If you want, I can expand on any of these or suggest practical routines and tools to put them into practice.

Best,
Your AI assistant


## Modern Tool Calling is implemented by the frameworks like OpenAI



In [16]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-5-mini",  # use an actual available OpenAI model
    messages=[
        {"role": "user", "content": "Hi, send an email to lucasbnsoares@hotmail.com, containing 3 things you think are essential for someone to be a life long learner"}
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "send_email",
                "description": "Send an email to the user",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "to": {"type": "string", "description": "Recipient email address"},
                        "subject": {"type": "string", "description": "Email subject line"},
                        "body": {"type": "string", "description": "Email body content"}
                    },
                    "required": ["to", "subject", "body"]
                }
            }
        }
    ]
)

response

ChatCompletion(id='chatcmpl-D5G88sAjZJar4nRMAgJKBCLNZP8nZ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_ye8pTtG4TAbKyKsuWzFsUcyu', function=Function(arguments='{"to":"lucasbnsoares@hotmail.com","subject":"Three essentials for becoming a lifelong learner","body":"Hi Lucas,\\n\\nHere are three things I think are essential for someone to be a lifelong learner:\\n\\n1) Curiosity and a growth mindset\\n   - Stay curious and treat challenges as opportunities to grow. Believe that abilities can be developed with effort, and view mistakes as learning signals rather than failures.\\n\\n2) Regular, deliberate practice and reflection\\n   - Build consistent learning habits (short daily study, projects, or exercises). Focus on deliberate practice—set specific goals, seek feedback, and reflect o

In [ ]:
# Parse tool calls from the OpenAI response object (OpenAI v1+)
tool_calls = response.choices[0].message.tool_calls

for tool_call in tool_calls:
    print(f"Tool call type: {tool_call.type}")
    print(f"Function name: {tool_call.function.name}")
    print(f"Function arguments: {tool_call.function.arguments}")

Tool call type: function
Function name: send_email
Function arguments: {"to":"lucasbnsoares@hotmail.com","subject":"Three essentials for becoming a lifelong learner","body":"Hi Lucas,\n\nHere are three things I think are essential for someone to be a lifelong learner:\n\n1) Curiosity and a growth mindset\n   - Stay curious and treat challenges as opportunities to grow. Believe that abilities can be developed with effort, and view mistakes as learning signals rather than failures.\n\n2) Regular, deliberate practice and reflection\n   - Build consistent learning habits (short daily study, projects, or exercises). Focus on deliberate practice—set specific goals, seek feedback, and reflect on what worked and what didn’t to improve steadily.\n\n3) Information literacy and adaptability\n   - Learn to find, evaluate, and synthesize reliable information. Be open to new tools and perspectives so you can adjust your methods as knowledge and technology evolve.\n\nBest regards,\nYour AI assistant"

## Tool Call = process of parsing/extracting LLM outputs into executable commands that run code outside the scope of the LLM